# Custom multi-action agent

This notebook goes through how to create your own custom agent.

An agent consists of two parts:

- Tools: The tools the agent has available to use.
- The agent class itself: this decides which action to take.
        
        
In this notebook we walk through how to create a custom agent that predicts/takes multiple steps at a time.

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [22]:
from langchain.agents import AgentExecutor, BaseMultiActionAgent, Tool
from langchain_community.utilities import SerpAPIWrapper

In [3]:
def random_word(query: str) -> str:
    print("\nNow I'm doing this!")
    return "foo"

In [4]:
import os

In [5]:
from google.colab import userdata
SERPAPI_API_KEY=userdata.get('SERPAPI_API_KEY')

In [6]:
os.environ["SERPAPI_API_KEY"] = "da17c0ef2d2f8363db481dace3c36bb05e06a9e4971028ec16bd4c98ede24c0d"

In [16]:
pip install google-search-results

In [8]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="RandomWord",
        func=random_word,
        description="call this to get a random word.",
    ),
]

In [9]:
from typing import Any, List, Tuple, Union

from langchain_core.agents import AgentAction, AgentFinish


class FakeAgent(BaseMultiActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

In [28]:
agent = FakeAgent()

In [11]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [23]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
In 2023, Canada's population jumped by over 1 million people for the first time in the country's history. The population now stands at 39.5 million and is set to pass the 40 million mark later this year.
Now I'm doing this!
foo

> Finished chain.


'bar'

In [13]:
agent_executor.run("How many people live in India as of 2023?")



> Entering new AgentExecutor chain...
1,425,775,850
Now I'm doing this!
foo

> Finished chain.


'bar'

In [24]:
agent_executor.run("what is the capital of india?")



> Entering new AgentExecutor chain...
New Delhi
Now I'm doing this!
foo

> Finished chain.


'bar'

In [26]:
from typing import Any, List, Tuple, Union
from langchain_core.agents import AgentAction, AgentFinish

class CompanyInfoAgent(BaseMultiActionAgent):
    """Custom Agent for Handling Company Information."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decide what action to take.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        input_text = kwargs["input"].lower()  # Convert input text to lowercase for case-insensitive matching
        if "hr portal" in input_text:
            return AgentFinish(return_values={"output": "You can access the HR portal at [HR portal URL]."}, log="")
        elif "employee portal" in input_text:
            return AgentFinish(return_values={"output": "You can access the employee portal at [Employee portal URL]."}, log="")
        elif "company directory" in input_text:
            return AgentFinish(return_values={"output": "You can access the company directory at [Company directory URL]."}, log="")
        # Add more conditions for other company-related queries as needed
        else:
            return AgentFinish(return_values={"output": "I'm sorry, I couldn't understand your request."}, log="")


In [41]:
from typing import Any, List, Tuple, Union
#from langchain_core.agents import AgentAction, AgentFinish, BaseMultiActionAgent

class CompanyInfoAgent(BaseMultiActionAgent):
    """Custom Agent for Handling Company Information."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decide what action to take.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        input_text = kwargs["input"].lower()  # Convert input text to lowercase for case-insensitive matching
        if "hr portal" in input_text:
            return AgentFinish(return_values={"output": "You can access the portal Kubera for Hr related queries."}, log="")
        elif "employee portal" in input_text:
            return AgentFinish(return_values={"output": "You can access the employee portal Workday for all the queries."}, log="")
        elif "company directory" in input_text:
            return AgentFinish(return_values={"output": "You can access the company directory at [Company directory URL]."}, log="")
        # Add more conditions for other company-related queries as needed
        else:
            return AgentFinish(return_values={"output": "I'm sorry, I couldn't understand your request."}, log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decide what action to take asynchronously.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return self.plan(intermediate_steps=intermediate_steps, **kwargs)


In [42]:
# Instantiate the custom agent
agent = CompanyInfoAgent()



In [43]:
input_text = "How do I access the HR portal?"

result = agent.plan(intermediate_steps=[], input=input_text)

if isinstance(result, AgentFinish):
    print(result.return_values["output"])
else:
    # Handle unexpected result
    print("Unexpected result from agent.")

You can access the portal Kubera for Hr related queries.


In [44]:
input_text = "How do I access the employee portal?"

result = agent.plan(intermediate_steps=[], input=input_text)

if isinstance(result, AgentFinish):
    print(result.return_values["output"])
else:
    # Handle unexpected result
    print("Unexpected result from agent.")

You can access the employee portal Workday for all the queries.
